In [1]:
import pandas as pd

import warnings
import numpy as np
import re
import itertools
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px 
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cufflinks

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
from google.colab import drive #cell to run in colab
drive.mount('/content/drive')

data_path = r'/content/drive/MyDrive/DS IT Academy/Spotify/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# data_path = r'data/' #cell to run local

По факту максимально полезными для данного задания оказались только два файла, поэтому считаем только их.

In [4]:
data = pd.read_csv(data_path+"data.csv")
data_w_genres = pd.read_csv(data_path+'data_w_genres.csv')
data.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [5]:
data_w_genres.head()

,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
0,['show tunes'],"""Cats"" 1981 Original London Cast",0.590111,0.467222,250318.555556,0.394003,0.011400,0.290833,-14.448000,0.210389,117.518111,0.389500,38.333333,5,1,9
1,[],"""Cats"" 1983 Broadway Cast",0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,30.576923,5,1,26
2,[],"""Fiddler On The Roof” Motion Picture Chorus",0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.857143,0,1,7
3,[],"""Fiddler On The Roof” Motion Picture Orchestra",0.884926,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.851852,0,1,27
4,[],"""Joseph And The Amazing Technicolor Dreamcoat""...",0.510714,0.467143,270436.142857,0.488286,0.009400,0.195000,-10.236714,0.098543,122.835857,0.482286,43.000000,5,1,7


Поля с наименованиями песен и артистов засорены лишними символами, поэтому немного их очистим.
По факту списки это строки, а чтобы подтянуть жанры необходимо оперировать артистами по одному.

In [6]:
data_w_genres['genres'].values[0]

"['show tunes']"

In [7]:
data_w_genres['genres_upd'] = data_w_genres['genres'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])
data_w_genres['genres_upd'].values[0][0]

'show_tunes'

In [8]:
data['artists_upd_v1'] = data['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))
data['artists'].values[0]

"['Sergei Rachmaninoff', 'James Levine', 'Berliner Philharmoniker']"

In [9]:
data['artists_upd_v1'].values[0]

['Sergei Rachmaninoff', 'James Levine', 'Berliner Philharmoniker']

In [10]:
data[data['artists_upd_v1'].apply(lambda x: not x)]

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,artists_upd_v1
143,0.300,1921,0.772000,"[""Scarlet D'Carpio""]",0.560,249370,0.313,0,7b4eHImKQ51DYaQvNTdtEp,0.000005,6,0.1150,-8.346,0,Himno Nacional del Perú,0,1921-09-23,0.0376,107.501,[]
234,0.902,1923,0.994000,"[""King Oliver's Creole Jazz Band""]",0.708,194533,0.361,0,1xEEYhWxT4WhDQdxfPCT8D,0.883000,0,0.1030,-11.764,0,Snake Rag,20,1923,0.0441,105.695,[]
238,0.554,1923,0.996000,"[""King Oliver's Creole Jazz Band""]",0.546,170827,0.189,0,3rauXVLOOM5BlxWqUcDpkg,0.908000,0,0.3390,-15.984,1,Chimes Blues,13,1923,0.0581,80.318,[]
244,0.319,1923,0.995000,"[""Clarence Williams' Blue Five""]",0.520,197493,0.153,0,1UdqHVRFYMZKU2Q7xkLtYc,0.131000,0,0.3530,-14.042,1,Pickin' On Your Baby,11,1923,0.0440,102.937,[]
249,0.753,1923,0.994000,"[""King Oliver's Creole Jazz Band""]",0.359,187227,0.357,0,5SvyP1ZeJX1jA7AOZD08NA,0.819000,3,0.2900,-11.810,1,Tears,10,1923,0.0511,205.053,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169540,0.467,2015,0.000011,"[""Noel Gallagher's High Flying Birds""]",0.547,209480,0.827,0,5W5MRU12dpuOnd5WmWzian,0.000125,6,0.4090,-4.339,1,In The Heat Of The Moment,60,2015-03-02,0.0282,105.913,[]
169819,0.922,2016,0.027900,"[""Umphrey's McGee""]",0.812,270629,0.584,0,0gqp35lTH7mS2B6JDjYAdo,0.000127,0,0.0463,-9.266,1,Can't Rock My Dream Face,50,2016-11-11,0.0345,120.031,[]
169866,0.191,2017,0.086900,"[""Olivia O'Brien""]",0.788,208400,0.516,1,5648YJgWmV5TSYRdF6Cxl1,0.000000,9,0.1340,-8.533,0,Empty,58,2017-11-17,0.0743,118.986,[]
170031,0.353,2017,0.853000,"[""The Wailin' Jennys""]",0.474,222705,0.213,0,46Nz7uguhTyUWOXhRBbzxa,0.000003,11,0.2410,-11.677,1,Wildflowers,53,2017-10-27,0.0280,84.962,[]


Приводим поле артиста к нормальному виду, и после его будем использовать для обучения. Теперь если у песни несколько исполнителей, в поле artist_upd хранится список таких исполнителей.

In [11]:
data['artists_upd_v2'] = data['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
data['artists_upd'] = np.where(data['artists_upd_v2'].apply(lambda x: not x), data['artists_upd_v1'], data['artists_upd_v2'] )

В поле artists_song в одно строковое значение собираем исполнтелей песни и ее название.

In [12]:
data['artists_song'] = data.apply(lambda row: ' '.join(row['artists_upd'])+row['name'],axis = 1)
data.sort_values(['artists_song','release_date'], ascending = False, inplace = True)

In [13]:
data

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,...,mode,name,popularity,release_date,speechiness,tempo,artists_upd_v1,artists_upd_v2,artists_upd,artists_song
164470,0.1770,1989,0.568,['조정현'],0.447,237688,0.2150,0,2ghebdwe2pNXT4eL34T7pW,0.000001,...,1,그아픔까지사랑한거야,31,1989-06-15,0.0272,71.979,[조정현],[],[조정현],조정현그아픔까지사랑한거야
150072,0.3520,1992,0.381,['黑豹'],0.353,316160,0.6860,0,3KIuCzckjdeeVuswPo20mC,0.000000,...,1,DON'T BREAK MY HEART,35,1992-12-22,0.0395,200.341,[黑豹],[],[黑豹],黑豹DON'T BREAK MY HEART
159177,0.4580,1963,0.987,['黃國隆'],0.241,193480,0.0437,0,4prhqrLXYMjHJ6vpRAlasx,0.000453,...,1,藝旦調,23,1963-05-28,0.0443,85.936,[黃國隆],[],[黃國隆],黃國隆藝旦調
144488,0.7040,1963,0.771,['黃國隆'],0.610,208760,0.1750,0,6Pqs2suXEqCGx7Lxg5dlrB,0.000000,...,1,思想起,23,1963-05-28,0.0419,124.662,[黃國隆],[],[黃國隆],黃國隆思想起
159169,0.7400,1963,0.725,['黃國隆'],0.646,183946,0.2040,0,3tIPRVHChwXaJWUhhXeEwr,0.000003,...,0,小妹呀喂,23,1963-05-28,0.0389,113.046,[黃國隆],[],[黃國隆],黃國隆小妹呀喂
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170054,0.5520,2018,0.872,"['$NOT', 'SUS Valentino']",0.566,114547,0.5410,1,4yxFC6hIvXlD1YAtbuxcof,0.000038,...,1,Lovely,59,2018-03-25,0.2900,87.020,"[$NOT, SUS Valentino]",[],"[$NOT, SUS Valentino]",$NOT SUS ValentinoLovely
140789,0.3890,2020,0.336,"['$NOT', 'Night Lovell']",0.757,172253,0.6250,1,4zrYnAzX1CMRt9SoZdixMz,0.345000,...,1,Human (feat. Night Lovell),67,2020-05-22,0.1430,146.023,"[$NOT, Night Lovell]",[],"[$NOT, Night Lovell]",$NOT Night LovellHuman (feat. Night Lovell)
38549,0.0825,2020,0.606,"['$NOT', 'Maggie Lindemann']",0.713,187587,0.4700,1,3oGbHF3Kdwf3AsRCbBjUxu,0.000013,...,1,Moon & Stars (feat. Maggie Lindemann),76,2020-03-06,0.0864,84.518,"[$NOT, Maggie Lindemann]",[],"[$NOT, Maggie Lindemann]",$NOT Maggie LindemannMoon & Stars (feat. Maggi...
170607,0.6310,2020,0.169,"['$NOT', 'Flo Milli']",0.844,125853,0.7200,1,3PXi72ZtSqx1PZc40KS0Qj,0.000000,...,0,Mean,65,2020-10-30,0.3000,99.917,"[$NOT, Flo Milli]",[],"[$NOT, Flo Milli]",$NOT Flo MilliMean


Видим, что в датасете присутствуют неуникальные треки, поэтмоу удалим повторяющиеся значения.

In [14]:
data['artists_song'].value_counts()[data['artists_song'].value_counts()>1]

KAROL GA Ella                                                           11
Frédéric Chopin Vladimir HorowitzBallade No. 1 in G Minor, Op. 23       11
Modest Mussorgsky William KapellPictures at an Exhibition: Promenade     8
Daddy YankeeLimbo                                                        8
KAROL G OzunaHello                                                       7
                                                                        ..
Wes MontgomeryWest Coast Blues                                           2
ZZ TopLa Grange - 1999 Remaster                                          2
ABBAGimme! Gimme! Gimme! (A Man After Midnight)                          2
2PacTrapped                                                              2
B.B. KingYou Done Lost Your Good Thing Now                               2
Name: artists_song, Length: 11052, dtype: int64

In [15]:
data.drop_duplicates('artists_song',inplace = True)

In [16]:
data['artists_song'].value_counts()[data['artists_song'].value_counts()>1]

Series([], Name: artists_song, dtype: int64)

Создаем новый отдельный датасет для связки жанров по артисту с основным датасетом

In [17]:
artists_from_data = data[['artists_upd','id']].explode('artists_upd')

In [18]:
artists_from_data

,artists_upd,id
164470,조정현,2ghebdwe2pNXT4eL34T7pW
150072,黑豹,3KIuCzckjdeeVuswPo20mC
159177,黃國隆,4prhqrLXYMjHJ6vpRAlasx
144488,黃國隆,6Pqs2suXEqCGx7Lxg5dlrB
159169,黃國隆,3tIPRVHChwXaJWUhhXeEwr
...,...,...
140789,Night Lovell,4zrYnAzX1CMRt9SoZdixMz
38549,$NOT,3oGbHF3Kdwf3AsRCbBjUxu
38549,Maggie Lindemann,3oGbHF3Kdwf3AsRCbBjUxu
170607,$NOT,3PXi72ZtSqx1PZc40KS0Qj


In [19]:
artists_genres = artists_from_data.merge(data_w_genres, how = 'left', left_on = 'artists_upd',right_on = 'artists')
artists_genres_nonnull = artists_genres[~artists_genres.genres_upd.isnull()]
artists_genres_nonnull

,artists_upd,id,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,genres_upd
0,조정현,2ghebdwe2pNXT4eL34T7pW,['classic korean pop'],조정현,0.568000,0.447000,237688.000000,0.215000,0.000001,0.064900,-16.478000,0.027200,71.979000,0.177000,31.000000,10.0,1.0,2.0,[classic_korean_pop]
1,黑豹,3KIuCzckjdeeVuswPo20mC,"['chinese indie', 'chinese indie rock']",黑豹,0.381000,0.353000,316160.000000,0.686000,0.000000,0.056800,-9.103000,0.039500,200.341000,0.352000,35.000000,11.0,1.0,2.0,"[chinese_indie, chinese_indie_rock]"
2,黃國隆,4prhqrLXYMjHJ6vpRAlasx,[],黃國隆,0.785455,0.570818,174582.727273,0.148400,0.000083,0.142191,-21.610091,0.054355,119.586273,0.741273,23.000000,5.0,1.0,11.0,[]
3,黃國隆,6Pqs2suXEqCGx7Lxg5dlrB,[],黃國隆,0.785455,0.570818,174582.727273,0.148400,0.000083,0.142191,-21.610091,0.054355,119.586273,0.741273,23.000000,5.0,1.0,11.0,[]
4,黃國隆,3tIPRVHChwXaJWUhhXeEwr,[],黃國隆,0.785455,0.570818,174582.727273,0.148400,0.000083,0.142191,-21.610091,0.054355,119.586273,0.741273,23.000000,5.0,1.0,11.0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207359,Night Lovell,4zrYnAzX1CMRt9SoZdixMz,"['canadian hip hop', 'dark trap', 'emo rap', '...",Night Lovell,0.222175,0.769000,190678.680000,0.553360,0.074681,0.129424,-8.043200,0.063016,118.288120,0.201076,63.400000,1.0,1.0,25.0,"[canadian_hip_hop, dark_trap, emo_rap, ottawa_..."
207360,$NOT,3oGbHF3Kdwf3AsRCbBjUxu,"['emo rap', 'florida rap', 'sad rap', 'undergr...",$NOT,0.544467,0.789800,137910.466667,0.532933,0.023063,0.180300,-9.149267,0.293687,112.344800,0.480700,67.533333,1.0,1.0,15.0,"[emo_rap, florida_rap, sad_rap, underground_hi..."
207361,Maggie Lindemann,3oGbHF3Kdwf3AsRCbBjUxu,"['dance pop', 'pop', 'post-teen pop', 'social ...",Maggie Lindemann,0.462571,0.611571,207565.285714,0.597143,0.018864,0.228571,-5.467857,0.079957,113.759429,0.304571,69.000000,4.0,1.0,7.0,"[dance_pop, pop, post-teen_pop, social_media_pop]"
207362,$NOT,3PXi72ZtSqx1PZc40KS0Qj,"['emo rap', 'florida rap', 'sad rap', 'undergr...",$NOT,0.544467,0.789800,137910.466667,0.532933,0.023063,0.180300,-9.149267,0.293687,112.344800,0.480700,67.533333,1.0,1.0,15.0,"[emo_rap, florida_rap, sad_rap, underground_hi..."


In [20]:
artists_genres_consolidated = artists_genres_nonnull.groupby('id')['genres_upd'].apply(list).reset_index()
artists_genres_consolidated

,id,genres_upd
0,000G1xMMuwxNHmwVsBdtj1,"[[candy_pop, dance_rock, new_wave, new_wave_po..."
1,000GyYHG4uWmlXieKLij8u,"[[alternative_hip_hop, conscious_hip_hop, minn..."
2,000Npgk5e2SgwGaIsN3ztv,"[[classic_bollywood, classic_pakistani_pop, fi..."
3,000jBcNljWTnyjB4YO7ojf,[[]]
4,000mGrJNc2GAgQdMESdgEc,"[[classical, french_romanticism, late_romantic..."
...,...,...
157290,7zyqJJ0QwajaHaPLCqZFXN,"[[electro_swing, nu_jazz, steampunk]]"
157291,7zywdG4ysfC5XNBzjQAo2o,"[[contemporary_country, country, country_road,..."
157292,7zzbPkvxJ6iANIxcSz60rm,"[[adult_standards, cool_jazz, jazz, lounge, sw..."
157293,7zzbfi8fvHe6hm342GcNYl,"[[blues_rock, country_rock, folk_rock, jam_ban..."


In [21]:
artists_genres_consolidated['consolidates_genre_lists'] = artists_genres_consolidated['genres_upd'].apply(lambda x: list(set(list(itertools.chain.from_iterable(x)))))

In [22]:
artists_genres_consolidated.head()

,id,genres_upd,consolidates_genre_lists
0,000G1xMMuwxNHmwVsBdtj1,"[[candy_pop, dance_rock, new_wave, new_wave_po...","[new_wave_pop, new_wave, candy_pop, rock, powe..."
1,000GyYHG4uWmlXieKLij8u,"[[alternative_hip_hop, conscious_hip_hop, minn...","[alternative_hip_hop, pop_rap, conscious_hip_h..."
2,000Npgk5e2SgwGaIsN3ztv,"[[classic_bollywood, classic_pakistani_pop, fi...","[sufi, ghazal, filmi, classic_pakistani_pop, c..."
3,000jBcNljWTnyjB4YO7ojf,[[]],[]
4,000mGrJNc2GAgQdMESdgEc,"[[classical, french_romanticism, late_romantic...","[historic_orchestral_performance, orchestral_p..."


Добавляем полученные данные по жанрам в основной датасет.

In [23]:
data = data.merge(artists_genres_consolidated[['id','consolidates_genre_lists']], on = 'id',how = 'left')

In [24]:
data

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,...,name,popularity,release_date,speechiness,tempo,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists
0,0.1770,1989,0.568,['조정현'],0.447,237688,0.2150,0,2ghebdwe2pNXT4eL34T7pW,0.000001,...,그아픔까지사랑한거야,31,1989-06-15,0.0272,71.979,[조정현],[],[조정현],조정현그아픔까지사랑한거야,[classic_korean_pop]
1,0.3520,1992,0.381,['黑豹'],0.353,316160,0.6860,0,3KIuCzckjdeeVuswPo20mC,0.000000,...,DON'T BREAK MY HEART,35,1992-12-22,0.0395,200.341,[黑豹],[],[黑豹],黑豹DON'T BREAK MY HEART,"[chinese_indie, chinese_indie_rock]"
2,0.4580,1963,0.987,['黃國隆'],0.241,193480,0.0437,0,4prhqrLXYMjHJ6vpRAlasx,0.000453,...,藝旦調,23,1963-05-28,0.0443,85.936,[黃國隆],[],[黃國隆],黃國隆藝旦調,[]
3,0.7040,1963,0.771,['黃國隆'],0.610,208760,0.1750,0,6Pqs2suXEqCGx7Lxg5dlrB,0.000000,...,思想起,23,1963-05-28,0.0419,124.662,[黃國隆],[],[黃國隆],黃國隆思想起,[]
4,0.7400,1963,0.725,['黃國隆'],0.646,183946,0.2040,0,3tIPRVHChwXaJWUhhXeEwr,0.000003,...,小妹呀喂,23,1963-05-28,0.0389,113.046,[黃國隆],[],[黃國隆],黃國隆小妹呀喂,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157660,0.2250,2020,0.164,"['$NOT', 'Wifisfuneral']",0.807,160731,0.5640,1,0lB24x4SNbi8lLhUAsdoMO,0.000000,...,BERETTA (feat. Wifisfuneral),69,2020-03-06,0.3190,96.004,"[$NOT, Wifisfuneral]",[],"[$NOT, Wifisfuneral]",$NOT WifisfuneralBERETTA (feat. Wifisfuneral),"[dark_trap, emo_rap, sad_rap, rap, vapor_trap,..."
157661,0.5520,2018,0.872,"['$NOT', 'SUS Valentino']",0.566,114547,0.5410,1,4yxFC6hIvXlD1YAtbuxcof,0.000038,...,Lovely,59,2018-03-25,0.2900,87.020,"[$NOT, SUS Valentino]",[],"[$NOT, SUS Valentino]",$NOT SUS ValentinoLovely,"[emo_rap, sad_rap, vapor_trap, underground_hip..."
157662,0.3890,2020,0.336,"['$NOT', 'Night Lovell']",0.757,172253,0.6250,1,4zrYnAzX1CMRt9SoZdixMz,0.345000,...,Human (feat. Night Lovell),67,2020-05-22,0.1430,146.023,"[$NOT, Night Lovell]",[],"[$NOT, Night Lovell]",$NOT Night LovellHuman (feat. Night Lovell),"[canadian_hip_hop, dark_trap, emo_rap, ottawa_..."
157663,0.0825,2020,0.606,"['$NOT', 'Maggie Lindemann']",0.713,187587,0.4700,1,3oGbHF3Kdwf3AsRCbBjUxu,0.000013,...,Moon & Stars (feat. Maggie Lindemann),76,2020-03-06,0.0864,84.518,"[$NOT, Maggie Lindemann]",[],"[$NOT, Maggie Lindemann]",$NOT Maggie LindemannMoon & Stars (feat. Maggi...,"[emo_rap, dance_pop, social_media_pop, sad_rap..."


Запишем в поле год реальный год выпуска трека

In [25]:
data['year'] = data['release_date'].apply(lambda x: int(x.split('-')[0]))

In [26]:
data['popularity'] = data.popularity.astype('float64') #преобразуем в float64 значение, чтобы отшкалировать с другими числовыми данными

In [27]:
float_cols = data.dtypes[data.dtypes == 'float64'].index.values
float_cols

array(['valence', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'popularity',
       'speechiness', 'tempo'], dtype=object)

In [28]:
data['consolidates_genre_lists'] = data['consolidates_genre_lists'].apply(lambda d: d if isinstance(d, list) else [])

Здесь выделяю колонки, которые позже буду использовать для отображения результата и которые несут смысл для пользователя.

In [29]:
cols_to_show = ['artists_upd', 'name', 'release_date', 'consolidates_genre_lists']

In [30]:
def ohe_prep(df, column, new_name): 

    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df


In [31]:
def create_feature_set(df, float_cols):
    
    #tfidf
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['consolidates_genre_lists'].apply(lambda x: " ".join(x)))
    print(tfidf_matrix.shape)
    genre_df = pd.DataFrame(tfidf_matrix.toarray(), dtype = 'float32')
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
    genre_df.reset_index(drop = True, inplace=True)
  
    year_ohe = ohe_prep(df, 'year','year').astype('float32')

    #шкалирование
    floats = df[float_cols].reset_index(drop = True)
    scaler = StandardScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns)

    final = pd.concat([genre_df, floats_scaled, year_ohe], axis = 1)
     
    final['id']=df['id'].values
    
    return final

In [32]:
complete_feature_set = create_feature_set(data, float_cols=float_cols)

(157665, 2938)


Получаем датасет, который будем использовать для рекомендаций.

In [33]:
complete_feature_set

,genre|21st_century_classical,genre|432hz,genre|_hip_hop,genre|a_cappella,genre|abstract,genre|abstract_beats,genre|abstract_hip_hop,genre|accordeon,genre|accordion,genre|acid_house,...,year|2012,year|2013,year|2014,year|2015,year|2016,year|2017,year|2018,year|2019,year|2020,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2ghebdwe2pNXT4eL34T7pW
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3KIuCzckjdeeVuswPo20mC
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4prhqrLXYMjHJ6vpRAlasx
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6Pqs2suXEqCGx7Lxg5dlrB
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3tIPRVHChwXaJWUhhXeEwr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0lB24x4SNbi8lLhUAsdoMO
157661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4yxFC6hIvXlD1YAtbuxcof
157662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4zrYnAzX1CMRt9SoZdixMz
157663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3oGbHF3Kdwf3AsRCbBjUxu


Получаем одну запись из датасета, для которой будем искать N похожих треков.

In [34]:
track = data.sample(1)
track

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,...,name,popularity,release_date,speechiness,tempo,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists
104565,0.287,1964,0.129,['Gil Evans'],0.301,395293,0.459,0,1IEbtXrHgN37lTQi9Byx6g,0.0166,...,Las Vegas Tango,37.0,1964-01-01,0.0365,95.846,[Gil Evans],[],[Gil Evans],Gil EvansLas Vegas Tango,"[bebop, cool_jazz, hard_bop, contemporary_post..."


Здесь исходя из выбранной для подбора рекомендаций песни разделяем датасет с векторами на элемент выбранного трека и весь остальной. Во имя возможности хоть что-то посчитать, датасет с остальными треками пришлось урезать рандомными 50000 значениями, иначе в память не лезет(

In [35]:
def generate_playlist_feature(complete_feature_set, playlist_df):

    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'] == playlist_df['id'].values[0]].drop('id', axis=1)
    complete_feature_set_nonplaylist = complete_feature_set[~(complete_feature_set['id'] == playlist_df['id'].values[0])]
    
    return complete_feature_set_playlist, complete_feature_set_nonplaylist.sample(50000)

In [36]:
complete_feature_set_playlist_vector_EDM, complete_feature_set_nonplaylist_EDM = generate_playlist_feature(complete_feature_set, track)

In [37]:
complete_feature_set_playlist_vector_EDM.shape

(1, 3048)

Считаем косинусное расстояние между выбранным треком и всеми остальными и выбираем N похожих

In [38]:
def generate_playlist_recos(df, features, nonplaylist_features, num_rec):

    nonplaylist_features['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values)[:,0]
    non_playlist_df_top = nonplaylist_features.sort_values('sim',ascending = False).head(num_rec)
    
    return non_playlist_df_top

In [39]:
edm_top_10 = generate_playlist_recos(data, complete_feature_set_playlist_vector_EDM, complete_feature_set_nonplaylist_EDM, 10)

Маленькая функция для печати результата

In [40]:
def print_result(df):
  for index, row in df.iterrows():
    print('{} with song "{}" released at {}. \nGenres: {}'.format(''.join(row['artists_upd']), row['name'], row['release_date'], ', '.join(row['consolidates_genre_lists'])))
    print()

In [41]:
print_result(track[cols_to_show])

Gil Evans with song "Las Vegas Tango" released at 1964-01-01. 
Genres: bebop, cool_jazz, hard_bop, contemporary_post-bop, avant-garde_jazz, jazz, free_jazz, soul_jazz



In [42]:
id_rec = edm_top_10['id'].to_list()

print_result(data[data['id'].isin(id_rec)][cols_to_show])

Walter Hawkins with song "Be Grateful" released at 1993-08-17. 
Genres: gospel

Venom with song "Buried Alive" released at 1982-11-01. 
Genres: hard_rock, speed_metal, old_school_thrash, death_metal, metal, nwobhm, rock, black_metal, thrash_metal, british_black_metal, black_thrash

The Beach Boys with song "Don't Talk (Put Your Head On My Shoulder) - Remastered" released at 1966-06-16. 
Genres: classic_rock, sunshine_pop, psychedelic_rock, brill_building_pop, rock, folk_rock, baroque_pop

Styx with song "Snowblind" released at 1981-01-01. 
Genres: classic_rock, hard_rock, progressive_rock, soft_rock, rock, folk_rock, art_rock, mellow_gold, album_rock

Styx with song "Clair De Lune / Ballerina" released at 1976-01-01. 
Genres: classic_rock, hard_rock, progressive_rock, soft_rock, rock, folk_rock, art_rock, mellow_gold, album_rock

Rev. Paul Jones with song "I Won’t Complain" released at 1996-01-01. 
Genres: praise, gospel

Renaissance with song "Can You Understand" released at 1973-01-0